In [1]:
from glob import glob
from streaming import MDSWriter
from streaming import StreamingDataset
from transformers import default_data_collator
from tqdm import tqdm
import numpy as np

In [2]:
!du -hs nanot5-*

410M	nanot5-0
213M	nanot5-1
265M	nanot5-10
2.6G	nanot5-11
533M	nanot5-12
527M	nanot5-13
1.1G	nanot5-14
679M	nanot5-15
542M	nanot5-16
407M	nanot5-17
527M	nanot5-18
279M	nanot5-19
346M	nanot5-2
562M	nanot5-20
279M	nanot5-21
860M	nanot5-22
563M	nanot5-23
523M	nanot5-24
556M	nanot5-25
765M	nanot5-26
2.8G	nanot5-27
2.7G	nanot5-28
277M	nanot5-29
850M	nanot5-3
715M	nanot5-4
274M	nanot5-5
213M	nanot5-6
270M	nanot5-7
269M	nanot5-8
678M	nanot5-9


In [3]:
folders = glob('nanot5-*')
folders

['nanot5-13',
 'nanot5-5',
 'nanot5-16',
 'nanot5-24',
 'nanot5-0',
 'nanot5-10',
 'nanot5-22',
 'nanot5-1',
 'nanot5-19',
 'nanot5-23',
 'nanot5-20',
 'nanot5-8',
 'nanot5-18',
 'nanot5-12',
 'nanot5-4',
 'nanot5-14',
 'nanot5-21',
 'nanot5-29',
 'nanot5-3',
 'nanot5-27',
 'nanot5-25',
 'nanot5-11',
 'nanot5-26',
 'nanot5-15',
 'nanot5-6',
 'nanot5-7',
 'nanot5-17',
 'nanot5-2',
 'nanot5-28',
 'nanot5-9']

In [4]:
from streaming.base.format.mds.encodings import Encoding, _encodings

class Int32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.int32)

_encodings['int32'] = Int32

In [5]:
columns = {
    'input_ids': 'int32',
}
compression = 'zstd'
hashes = 'sha1', 'xxh64'

In [6]:
!rm -rf nanot5

In [7]:
with MDSWriter(out='nanot5', columns=columns, compression=compression, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = StreamingDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|██████████| 199137/199137 [00:19<00:00, 10301.36it/s]


In [10]:
%%time
dataset = StreamingDataset(local='nanot5')

CPU times: user 62.8 ms, sys: 12.4 ms, total: 75.2 ms
Wall time: 74.1 ms


In [14]:
dataset[0]

{'input_ids': array([    1,   331, 11484, ...,  1515,    80,    17], dtype=int32)}

In [ ]:
model(**padded)